All of the data and metadata for the Allen Mouse Brain Atlas is now stored in a public bucket in Amazon's S3 data storage service. The utilities demonstrated here access that bucket through AWS's `boto3` API. In order to access the bucket, you will need an AWS account with credentials stored in a csv file that looks like
```
column1,Access key ID,Secret access key,column4,column5...
data1,your_access_key_id,your_secret_access_key,data2,...
```
The utilities in this module will use the credentials stored in that file to create a `boto3.Session` which will handle all transactions with S3.

We create that `Session` below, using the credentials in the file `accessKeys.csv`. This file is not a part of the repo you have cloned. It is a part of the developer's local system. You must provide your own equivalent of `accessKeys.csv` and alter the code below accordingly.

In [ ]:
import os
credential_filename = 'accessKeys.csv'
assert os.path.isfile(credential_filename)

import aws_utils
boto3_session = aws_utils.get_boto3_session(credential_filename)

Now that we have a `boto3.Session`, we will download the metadata for the Allen Mouse Brain Atlas. First, let's import the module containing the utilities for accessing the atlas.

In [ ]:
import aba_mouse_utils as mouse_utils

Now let's download the metadata for the entire atlas. The method below downloads the file into a directory `tmp/` which is a subdirectory of this repository. If you want to use a different temporary directory, you can set it with the `tmp_dir` kwarg in that method (and all of the methods below that download data from S3). In addition to downloading the metadata, the method below will also load it into memory, giving you access to a list of dicts, each of which represents one "section data set" in the Allen Mouse Brain Atlas.

In [ ]:
atlas_metadata = mouse_utils.get_atlas_metadata(session=boto3_session)

Consider just the first two elements in `atlas_metadata`

In [ ]:
atlas_metadata[:2]

Here we can see the very high level metadata about each of the section data sets in the atlas. To download the detailed metadata about one of these sets, we use the class `SectionDatSet`, which we instantiate with the `id` of the section data set found in the atlas metadata above.

In [ ]:
section_data = mouse_utils.SectionDataSet(5, session=boto3_session)

`section_data.metadata` is now a dict containing all of the metadata about the section data set.

In [ ]:
key_list = list(section_data.metadata.keys())
key_list.sort()
print(key_list)

In [ ]:
print(section_data.metadata)

Metadata about the images that make up this section dataset can be accessed according to either their `sub_image_id` or their `tissue_index`. Below, we show lists of valid values for these parameters.

In [ ]:
print(section_data.tissue_indices)
print('')
print(section_data.sub_image_ids)

To get the metadata associated with one of these images, use

In [ ]:
image_metadata = section_data.image_metadata_from_tissue_index(58)

In [ ]:
print(image_metadata['id'])

In [ ]:
alt_metadata = section_data.image_metadata_from_sub_image(image_metadata['id'])
assert alt_metadata == image_metadata

In [ ]:
print(image_metadata.keys())

All of the images in the atlas exist at multiple resolutions. Each resolution is identified by a downsampling index. `downsampling == 0` corresponds to the full resolution image. Each larger downsampling index corresponds to a factor of 2 reduction in resolution in both the x and y direction, so `downsampling == 1` is a factor of 4 smaller than `downsampling == 0`; `downsampling == 2` is a factor of 16 smaller than `downsampling == 0`; etc.

To see what levels of downsampling are available for each image, use

In [ ]:
downsampling_indexes = list(image_metadata['downsampling'].keys())
downsampling_indexes.sort()
print(downsampling_indexes)

**Note:** in some rare cases, the full-resolution image file became corrupted during processing and was lost. These will correspond to images that have a valid `downsample_1` but no `downsample_0`. If you ever ask for a downsampling index that does not exist, the code should fail gracefully and issue a warning.

The methods `SectionDataSet.download_image_from_tissue_index` and `SectionDataSet.download_image_from_sub_image` allow you to download an image as a TIFF file, specifying it by either its tissue index or its sub-image ID and its downsampling index.

Let's download the `downsampling == 3` copy of the image with `tissue_index == 58`. We must specify a local filename in which to write the image.

These methods return `True` if the image was downloaded; `False` otherwise.

In [ ]:
local_filename = 'example_aba_image.tiff'
section_data.download_image_from_tissue_index(58, 3, local_filename)

Let's visualize the image with matplotlib.

In [ ]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

import PIL.Image

In [ ]:
plt.figure(figsize=(15,15))
img = PIL.Image.open(local_filename)
plt.imshow(img)